In [5]:
import psycopg2
import pandas as pd
import numpy as np

In [6]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="postgres"
)

In [7]:
cur = conn.cursor()

In [8]:
cur.execute("""
        select bank_id, time_internal, wight, swtc+swtv total
        from(
            select bank_id, time_internal, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
            from(
                select bank_id, time_internal, sum(new_count) wight, transaction_cost, transaction_value
                from(
                    select *
                    from(
                        select bank_id, time_internal,  count(count) new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count = 0
                        group by bank_id, time_internal, transaction_cost, transaction_value) as b0
 
                    union all
                    select *
                    from(
                        select bank_id, time_internal,  count new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count != 0) as b1) as b2
                group by bank_id, time_internal, transaction_cost, transaction_value) as b3
            group by bank_id, time_internal) as b4
            where bank_id = 13335
            order by total desc""")

In [9]:
result = cur.fetchall()
pd.DataFrame(result)

,0,1,2,3
0,13335,2023-07-14 19:00:00+00:00,87535210435,1219891


In [10]:
data = pd.read_csv('tranasaction_dataset.csv', parse_dates=['time_internal'])

In [11]:
tmp_data = data[data['count'] == 0]
dg1 = tmp_data.groupby(by=['bank_id', 'time_internal', 'transaction_cost', 'transaction_value']).agg({'count': 'count'}).reset_index()

In [12]:
tmp_data2 = data[data['count'] != 0]
dg2 = tmp_data2[['count', 'bank_id', 'time_internal', 'transaction_cost', 'transaction_value']]

In [13]:
dg = pd.concat([dg1, dg2])

In [14]:
dg = dg.groupby(by=['bank_id', 'time_internal', 'transaction_cost', 'transaction_value']).agg({'count': 'sum'}).reset_index()

In [15]:
dg = dg.groupby(by=['bank_id', 'time_internal']).agg({
    'count' : 'sum',
    'transaction_cost' : 'sum',
    'transaction_value' : 'sum'
}).reset_index()

In [16]:
dg['total_value'] = dg['transaction_cost'] + dg['transaction_value']
dg = dg.drop(['transaction_cost', 'transaction_value'], axis='columns')

In [17]:
your_bank_id = 13335
dg[dg['bank_id'] == your_bank_id].sort_values('total_value', ascending=False)

,bank_id,time_internal,count,total_value
1301,13335,2023-07-14 19:00:00+00:00,1219891,87535210435
